In [1]:
#import the module
import json
import pandas as pd
import requests
import numpy as np

#retrieve theMovieDB.org API key
def get_keys(fname):
    '''
    Init signature: get_keys(filename)
    Docstring:     
    get_keys(filename='') -> dict
    Retrieve API key stored on file.  return a dictionary.
    
    Parameters
    ----------
    filename: Loation of the json file where the API key is stored.
      
    '''
    with open(fname) as f:
        return json.load(f)


In [2]:
# import the api key for movieDB using the get_keys function
keys = get_keys('moviedb.json')

#assign API value to variabloe
api_key = keys['api_key']


In [46]:
# Use mtdb API to retrieve list of movies with primary_release_date greater than 01-01-2010
def get_mtdb_movies():
    '''
    init signature:
        get_mtdb_movies() 
    docstring:
        get_mtdb_movies() -> dataframe
        Using TMDB API to retrive a list of movies
    '''
    # Define get request parameters
    url = 'https://api.themoviedb.org/3/discover/movie?'
    langauge = 'en-US'
    sort_by = 'popularity.desc'
    include_adult = 'false'
    include_video = 'false'
    primary_release_date_gt = '2010-01-01'

    url_param = {
                    'api_key': api_key,
                    'language': langauge,
                    'sort_by': sort_by,
                    'include_adult': include_adult,
                    'include_video': include_video,
                    'primary_release_date.gt': primary_release_date_gt
                }

# API only return 1 page with 20 results.  Set the loop to 500 times since the max number
# of pages allowed by TMDB is 500.
    for i in range(1,500): 
        # update the page number using the iteration value and pass it to the request parameter
        url_param.update({'page': i})
#         print('page', i)
        # create the response ojb.
        resp = requests.get(url, params=url_param)
        # if request status is sucessful and the value of key ['page'] is 1
        if resp.status_code == 200 and resp.json()['page'] == 1:
            # create the DataFrame
            df = pd.DataFrame.from_dict(resp.json()['results'])
        # if request status is sucessful and able to read the value of key ['page']
        elif resp.status_code == 200 and resp.json().get('page', 10000) < 10000:
            # convert response to dataframe
            a = pd.DataFrame.from_dict(resp.json()['results'])
            # concat the present df with the previous pass
            df = pd.concat([df, a], sort=False)
        # if request status is sucessful but unable to read the value of key ['page'],
        # assign value 10,000 and quit the loop  
        elif resp.status_code == 200 and resp.json().get('page', 10000) == 10000:
            break
    
    return df


In [ ]:
# Retrieve movie list from TMDB via their API
df = get_mtdb_movies()


In [47]:
# Check df
df.head()

,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,1305.559,591,False,/uPGq1mkEXznUpapDmOSxbsybjfp.jpg,475303,False,/6fkqwqLEcDZOEAnBBfKAniwNxtx.jpg,en,A Rainy Day in New York,"[35, 10749]",A Rainy Day in New York,6.7,Two young people arrive in New York to spend a...,2019-07-26
1,271.489,2483,False,/xBHvZcjRiWyobQ9kxBhO6B2dtRI.jpg,419704,False,/5BwqwxMEjeFtdknRV792Svo0K1v.jpg,en,Ad Astra,"[12, 18, 9648, 878, 53]",Ad Astra,6.0,"The near future, a time when both hope and har...",2019-09-17
2,217.757,546,False,/aQvJ5WPzZgYVDrxLX4R6cLJCEaQ.jpg,454626,False,/qonBhlm0UjuKX2sH7e73pnG0454.jpg,en,Sonic the Hedgehog,"[28, 35, 878, 10751]",Sonic the Hedgehog,7.1,Based on the global blockbuster videogame fran...,2020-02-12
3,272.710,1901,False,/bB42KDdfWkOvmzmYkmK58ZlCa9P.jpg,512200,False,/hreiLoPysWG79TsyQgMzFKaOTF5.jpg,en,Jumanji: The Next Level,"[28, 12, 35, 14]",Jumanji: The Next Level,6.8,As the gang return to Jumanji to rescue one of...,2019-12-04
4,215.930,174,False,/4U7hpTK0XTQBKT5X60bKmJd05ha.jpg,570670,False,/uZMZyvarQuXLRqf3xdpdMqzdtjb.jpg,en,The Invisible Man,"[27, 9648, 878, 53]",The Invisible Man,7.5,When Cecilia's abusive ex takes his own life a...,2020-02-26


In [49]:
# Check the dimension of the df
df.shape

(9980, 14)

In [48]:
# Export initial df to csv since it is a lenghty process
df.to_csv('raw_tmdb.csv')

In [53]:
# Reset df index.  This will remove the duplicate index cause by concatenating the df's
df2 = df.reset_index()

In [56]:
# Check df2
df2.head()

,index,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,0,1305.559,591,False,/uPGq1mkEXznUpapDmOSxbsybjfp.jpg,475303,False,/6fkqwqLEcDZOEAnBBfKAniwNxtx.jpg,en,A Rainy Day in New York,"[35, 10749]",A Rainy Day in New York,6.7,Two young people arrive in New York to spend a...,2019-07-26
1,1,271.489,2483,False,/xBHvZcjRiWyobQ9kxBhO6B2dtRI.jpg,419704,False,/5BwqwxMEjeFtdknRV792Svo0K1v.jpg,en,Ad Astra,"[12, 18, 9648, 878, 53]",Ad Astra,6.0,"The near future, a time when both hope and har...",2019-09-17
2,2,217.757,546,False,/aQvJ5WPzZgYVDrxLX4R6cLJCEaQ.jpg,454626,False,/qonBhlm0UjuKX2sH7e73pnG0454.jpg,en,Sonic the Hedgehog,"[28, 35, 878, 10751]",Sonic the Hedgehog,7.1,Based on the global blockbuster videogame fran...,2020-02-12
3,3,272.710,1901,False,/bB42KDdfWkOvmzmYkmK58ZlCa9P.jpg,512200,False,/hreiLoPysWG79TsyQgMzFKaOTF5.jpg,en,Jumanji: The Next Level,"[28, 12, 35, 14]",Jumanji: The Next Level,6.8,As the gang return to Jumanji to rescue one of...,2019-12-04
4,4,215.930,174,False,/4U7hpTK0XTQBKT5X60bKmJd05ha.jpg,570670,False,/uZMZyvarQuXLRqf3xdpdMqzdtjb.jpg,en,The Invisible Man,"[27, 9648, 878, 53]",The Invisible Man,7.5,When Cecilia's abusive ex takes his own life a...,2020-02-26


In [59]:
# Remove irrelevent columns (data cleaning process)
df2.drop(['index', 'video', 'poster_path', 'adult', 'backdrop_path', 'overview'], axis=1, inplace=True)


In [60]:
# Checking df2
df2.head()

,popularity,vote_count,id,original_language,original_title,genre_ids,title,vote_average,release_date
0,1305.559,591,475303,en,A Rainy Day in New York,"[35, 10749]",A Rainy Day in New York,6.7,2019-07-26
1,271.489,2483,419704,en,Ad Astra,"[12, 18, 9648, 878, 53]",Ad Astra,6.0,2019-09-17
2,217.757,546,454626,en,Sonic the Hedgehog,"[28, 35, 878, 10751]",Sonic the Hedgehog,7.1,2020-02-12
3,272.710,1901,512200,en,Jumanji: The Next Level,"[28, 12, 35, 14]",Jumanji: The Next Level,6.8,2019-12-04
4,215.930,174,570670,en,The Invisible Man,"[27, 9648, 878, 53]",The Invisible Man,7.5,2020-02-26


In [64]:
# Review the column names
df2.columns

Index(['popularity', 'vote_count', 'id', 'original_language', 'original_title',
       'genre_ids', 'title', 'vote_average', 'release_date'],
      dtype='object')

In [146]:
# Check for row duplication (data cleaning process).  Exclude column genre_ids becuase it is a list
df2.duplicated(subset=['popularity', 'vote_count', 'id',
                       'original_language', 'original_title',
                       'title', 'vote_average', 'release_date']).sum()

0

In [61]:
# Check for duplicate title (data cleaning process)
df2.title.duplicated().sum()

316

In [69]:
# Investigate the duplicate titles.  Preview of the duplicate titles
df2.loc[df2.title.duplicated(),:]

,popularity,vote_count,id,original_language,original_title,genre_ids,title,vote_average,release_date
111,30.211,7417,812,en,Aladdin,"[12, 16, 14, 10749, 10751]",Aladdin,7.6,1992-11-25
118,31.593,11693,8587,en,The Lion King,"[16, 18, 10751]",The Lion King,8.3,1994-05-07
198,29.431,6448,10020,en,Beauty and the Beast,"[16, 14, 10749, 10751]",Beauty and the Beast,7.7,1991-11-13
212,18.464,2564,2907,en,The Addams Family,"[35, 14]",The Addams Family,7.0,1991-11-22
287,22.063,2638,4327,en,Charlie's Angels,"[28, 12, 35, 80, 53]",Charlie's Angels,5.7,2000-11-02
...,...,...,...,...,...,...,...,...,...
9845,7.510,138,44678,en,Altitude,"[27, 28, 53, 878]",Altitude,4.9,2010-10-10
9871,8.505,394,17529,en,True Grit,"[18, 37]",True Grit,7.4,1969-06-11
9929,5.684,31,32528,en,Spartacus,"[28, 18, 36, 10770]",Spartacus,7.0,2004-03-17
9935,8.754,221,38780,en,Rampage,"[28, 80, 18, 9648, 53]",Rampage,6.0,2009-08-14


In [72]:
# Spot checking the titles.  They appear to be re-make.  Will keep the records.
df2[(df2['title'] == 'Aladdin') | (df2['title'] == 'Rampage') | (df2['title'] == 'The Addams Family')]

,popularity,vote_count,id,original_language,original_title,genre_ids,title,vote_average,release_date
40,50.073,601,481084,en,The Addams Family,"[16, 35, 14, 10751]",The Addams Family,6.2,2019-10-10
42,46.144,4930,420817,en,Aladdin,"[12, 35, 14, 10749, 10751]",Aladdin,7.0,2019-05-22
111,30.211,7417,812,en,Aladdin,"[12, 16, 14, 10749, 10751]",Aladdin,7.6,1992-11-25
212,18.464,2564,2907,en,The Addams Family,"[35, 14]",The Addams Family,7.0,1991-11-22
724,19.886,3745,427641,en,Rampage,"[28, 12, 14, 878]",Rampage,6.2,2018-04-12
9935,8.754,221,38780,en,Rampage,"[28, 80, 18, 9648, 53]",Rampage,6.0,2009-08-14


In [79]:
# Check for nan value (data cleaning process)
df2.isnull().sum()

popularity           0
vote_count           0
id                   0
original_language    0
original_title       0
genre_ids            0
title                0
vote_average         0
release_date         4
dtype: int64

In [80]:
df2.loc[df2.release_date.isnull(), :]

,popularity,vote_count,id,original_language,original_title,genre_ids,title,vote_average,release_date
1333,17.487,0,504075,zh,战狼3,"[28, 10752]",Wolf Warrior 3,0.0,NaN
4873,5.838,0,425274,en,Now You See Me 3,"[80, 53]",Now You See Me 3,0.0,NaN
6421,5.954,0,374771,en,Riddick 4: Furya,"[28, 878]",Riddick 4: Furya,0.0,NaN
9056,4.547,0,425770,ta,சதுரங்க வேட்டை 2,"[28, 80, 53]",Sathuranga Vettai 2,0.0,NaN


In [94]:
# Since we are intertested in 'original_language' = en, we will drop zh and ta
# lookk up the release the release_date for the other two online.

# Remove row 1333 & 9056
df2.drop(index=[1333, 9056], axis=0, inplace=True)

In [95]:
# Check that zh & ta are in fact removed
df2.loc[df2.release_date.isnull(), :]

,popularity,vote_count,id,original_language,original_title,genre_ids,title,vote_average,release_date
4873,5.838,0,425274,en,Now You See Me 3,"[80, 53]",Now You See Me 3,0.0,NaN
6421,5.954,0,374771,en,Riddick 4: Furya,"[28, 878]",Riddick 4: Furya,0.0,NaN


In [98]:
#  Both 4873 and 6421 do not have a release date set in 2020.  Remove from df.
df2.drop(index=[4873, 6421], axis=0, inplace=True)


In [99]:
# Re check isnull status
df2.isnull().sum()

popularity           0
vote_count           0
id                   0
original_language    0
original_title       0
genre_ids            0
title                0
vote_average         0
release_date         0
dtype: int64

In [104]:
# create a function to send request to pull movie details.  Interested in the revenue and the budget values
def get_movie_bud_rev(movieid,rev_bud='revenue'):
    '''
    Init signature:
        get_movie_bud_rev(movie_id, rev_bud='revenue')
    Docstring:     
        get_movie_bud_rev(movie_id, rev_bud='revenue') -> dict
        Retrieve the revenue info from the reponse object
    
    Parameters
    ----------
    movie_id: TMDB movie id as int
    rev_bud: dictionary key to search in the response object

    '''
    # define get request parameters
    url = 'https://api.themoviedb.org/3/movie/' + str(movieid) + '?'
    url_param = {
                    'api_key': api_key,
                    'language': 'en-US',
                }
    resp = requests.get(url, params=url_param)
    if resp.status_code == 200:
        resp_bud_rev = resp.json()[rev_bud]
#         print(movieid)
    else:
        resp_bud_rev = np.nan
  
        
    return resp_bud_rev


In [ ]:
# Create the revenue columns
df2['revenue'] = df2['id'].apply(lambda x: get_movie_bud_rev(x, 'revenue'))


In [ ]:
# Create the budget column
df2['budget'] = df2['id'].apply(lambda x: get_movie_bud_rev(x, 'budget'))

In [108]:
# Preview revenue & budget column 
df2.head()

,popularity,vote_count,id,original_language,original_title,genre_ids,title,vote_average,release_date,revenue,budget
0,1305.559,591,475303,en,A Rainy Day in New York,"[35, 10749]",A Rainy Day in New York,6.7,2019-07-26,0.0,0.0
1,271.489,2483,419704,en,Ad Astra,"[12, 18, 9648, 878, 53]",Ad Astra,6.0,2019-09-17,127175922.0,87500000.0
2,217.757,546,454626,en,Sonic the Hedgehog,"[28, 35, 878, 10751]",Sonic the Hedgehog,7.1,2020-02-12,265493652.0,85000000.0
3,272.710,1901,512200,en,Jumanji: The Next Level,"[28, 12, 35, 14]",Jumanji: The Next Level,6.8,2019-12-04,310830000.0,125000000.0
4,215.930,174,570670,en,The Invisible Man,"[27, 9648, 878, 53]",The Invisible Man,7.5,2020-02-26,0.0,9000000.0


In [112]:
# Check for NAN in revenue & budget coliumns
df2.revenue.isnull().sum(), df2.budget.isnull().sum()

(1, 1)

In [115]:
# Review the row to decide what to do
df2.loc[df2.revenue.isnull(), :]

,popularity,vote_count,id,original_language,original_title,genre_ids,title,vote_average,release_date,revenue,budget
2599,9.768,0,627494,en,When Calls the Heart: Home for Christmas,[],When Calls the Heart: Home for Christmas,0.0,2019-12-25,NaN,NaN


In [118]:
# Since the movie title is missing a lot of useful info, drop the row
df2.drop(index=[2599], axis=0, inplace=True)

In [119]:
# Get the genre id classification from tmdb web site
# for ref: https://api.themoviedb.org/3/genre/movie/list?api_key=api_key&language=en-US
def get_genre():
    '''
    Init signature:
        get_genre()
    Docstring:
        get_genre() -> dict
        
    '''
    # Create the response object
    g_resp = requests.get(
        'https://api.themoviedb.org/3/genre/movie/list?api_key='
        + str(api_key) + '&language=en-US'
        )
    # if get request is successful, store genres name in dict.
    if g_resp.status_code == 200:
        genre_dict =  sorted(g_resp.json()['genres'], key=lambda x: x['id'])
        
    return genre_dict
        

In [120]:
# map tmdb genre_id to genre_name
def match_genre_id(gid):
    '''
    Init signature:
        match_genre_id(gid)
    Docstring:
        match_genre_id(gid) -> list
        Take input genre id and match it to the "genre id:genre name" dict to retrieve the genre name
    Parameters:
        gid: The value in the genre_id column of the dataframe
    '''
    # Initiate an empty list to store the matching genre name
    genre_name = []
    # call get_genre function to pull the genre id/name from TMDB
    genre_dict = get_genre()
    # Match gerne id to pull gerne name
    for i in gid:
        for j in genre_dict:
            if j['id'] == i:
                genre_name.append(j['name'])
    
    return genre_name


In [122]:
# Create a new coluimn to store genre_name
df2['genre_names'] = df2['genre_ids'].apply(lambda x: match_genre_id(x))


In [125]:
# Took 20 min to run match_genre_id, create a backup
df2.to_csv('add_genre_name_tmdb.cdv')


In [ ]:
# Convert col release_date from string to datatime data type
import datetime

df2['release_date'] = pd.to_datetime(df2['release_date'])

# Note to self: above method is more useful compare to cnvert str yyyy-mm-dd to str yyyy
# df2['release_date'] = df2['release_date'].map(lambda x: x[0:4] if not pd.isnull(x) else x)


In [154]:
round(df2.revenue / 1000000,2)

0         0.00
1       127.18
2       265.49
3       310.83
4         0.00
         ...  
9975      0.27
9976      0.00
9977      0.00
9978      0.00
9979      0.00
Name: revenue, Length: 9975, dtype: float64

In [149]:
df2_yr = df2.groupby(pd.DatetimeIndex(df2.release_date).year)

In [152]:
df2_yr.revenue.describe()

,count,mean,std,min,25%,50%,75%,max
release_date,,,,,,,,
1896.0,1.0,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.000000e+00
1900.0,1.0,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.000000e+00
1902.0,1.0,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.000000e+00
1903.0,1.0,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.000000e+00
1915.0,1.0,1.100000e+07,NaN,11000000.0,11000000.0,11000000.0,11000000.0,1.100000e+07
...,...,...,...,...,...,...,...,...
2019.0,606.0,4.251287e+07,1.914641e+08,0.0,0.0,0.0,16284.0,2.797801e+09
2020.0,414.0,2.032698e+06,1.743986e+07,0.0,0.0,0.0,0.0,2.654937e+08
2021.0,13.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00


In [311]:
df3 = pd.read_csv('movieData.csv')

In [312]:
df3.head()

,Title,Year,Genre,Rating,Score,Director,Stars,Gross
0,Parasite,2019,"['Comedy', ' Drama', ' Thriller']",8.6,96.0,Bong Joon Ho,"['Kang-ho Song', 'Sun-kyun Lee', 'Yeo-jeong Jo...","$51,479,136"
1,The Invisible Man,2020,"['Horror', ' Mystery', ' Sci-Fi']",7.6,71.0,Leigh Whannell,"['Elisabeth Moss', 'Oliver Jackson-Cohen', 'Ha...","$28,205,665"
2,Knives Out,2019,"['Comedy', ' Crime', ' Drama']",8.0,82.0,Rian Johnson,"['Daniel Craig', 'Chris Evans', 'Ana de Armas'...","$164,655,243"
3,Sonic the Hedgehog,2020,"['Action', ' Adventure', ' Comedy']",6.8,47.0,Jeff Fowler,"['Ben Schwartz', 'James Marsden', 'Jim Carrey'...","$128,555,045"
4,Jojo Rabbit,2019,"['Comedy', ' Drama', ' War']",8.0,58.0,Taika Waititi,"['Roman Griffin Davis', 'Thomasin McKenzie', '...","$33,122,789"


In [342]:
df3.shape, df2.shape

((4786, 8), (8089, 9))

In [351]:
joined_df = df2.join(df3, how='outer')

In [352]:
joined_df.head()

,popularity,vote_count,vote_average,title,release_date,original_language,revenue,budget,genre_names,Title,Year,Genre,Rating,Score,Director,Stars,Gross
0,0.600,1.0,6.0,The Strange Cohabitation,2013,ko,NaN,NaN,"[Mystery, Horror]",Parasite,2019.0,"['Comedy', ' Drama', ' Thriller']",8.6,96.0,Bong Joon Ho,"['Kang-ho Song', 'Sun-kyun Lee', 'Yeo-jeong Jo...","$51,479,136"
1,5.511,66.0,7.5,A Brief History of Time,1991,en,0.0,0.0,[Documentary],The Invisible Man,2020.0,"['Horror', ' Mystery', ' Sci-Fi']",7.6,71.0,Leigh Whannell,"['Elisabeth Moss', 'Oliver Jackson-Cohen', 'Ha...","$28,205,665"
2,19.668,5053.0,7.3,American Psycho,2000,en,34266564.0,7000000.0,"[Thriller, Drama, Crime, Horror]",Knives Out,2019.0,"['Comedy', ' Crime', ' Drama']",8.0,82.0,Rian Johnson,"['Daniel Craig', 'Chris Evans', 'Ana de Armas'...","$164,655,243"
3,15.054,1177.0,7.5,Frida,2002,en,56298474.0,12000000.0,"[Drama, Romance]",Sonic the Hedgehog,2020.0,"['Action', ' Adventure', ' Comedy']",6.8,47.0,Jeff Fowler,"['Ben Schwartz', 'James Marsden', 'Jim Carrey'...","$128,555,045"
4,5.485,19.0,5.8,The Return of the King,1980,en,0.0,0.0,"[Animation, Fantasy, Family, TV Movie]",Jojo Rabbit,2019.0,"['Comedy', ' Drama', ' War']",8.0,58.0,Taika Waititi,"['Roman Griffin Davis', 'Thomasin McKenzie', '...","$33,122,789"


In [356]:
# joined_df['genre_names'] + joined_df['Genre']
type(joined_df['genre_names'][0]), type(joined_df['Genre'][0])

(list, str)

In [394]:
# 'Horror' in str(joined_df['genre_names'][0])
# str(joined_df['genre_names'][0]).find('Horror') != True
'Horror' in str(joined_df['genre_names'][0])

True

In [441]:
# df[df.City.str.contains('ville',case=False)].. joined_df.genre_names.str.contains('horror', case=False)
cond = (pd.notna(joined_df['genre_names']))
cond1 = ('Horror' in str(joined_df.genre_names))
cond2 = ['Horror']
# joined_df[joined_df.genre_names.str.contains('horror', case=False)]
nona = joined_df[cond]
nona.shape

(20, 17)

In [433]:
joined_df['genre_names'].head(10)                             

0                                    [Mystery, Horror]
1                                        [Documentary]
2                     [Thriller, Drama, Crime, Horror]
3                                     [Drama, Romance]
4               [Animation, Fantasy, Family, TV Movie]
5    [Adventure, Animation, Fantasy, Family, TV Movie]
6                    [Action, Drama, Western, Romance]
7                                     [Drama, Romance]
8                                     [Drama, Romance]
9                                              [Drama]
Name: genre_names, dtype: object